<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#背景" data-toc-modified-id="背景-1">背景</a></span></li><li><span><a href="#正文" data-toc-modified-id="正文-2">正文</a></span><ul class="toc-item"><li><span><a href="#基本概念" data-toc-modified-id="基本概念-2.1">基本概念</a></span><ul class="toc-item"><li><span><a href="#什么是RabbitMQ？" data-toc-modified-id="什么是RabbitMQ？-2.1.1">什么是RabbitMQ？</a></span></li><li><span><a href="#使用RabbitMQ后组件间如何通信？" data-toc-modified-id="使用RabbitMQ后组件间如何通信？-2.1.2">使用RabbitMQ后组件间如何通信？</a></span></li><li><span><a href="#Producer如何向RabbitMQ发送消息？" data-toc-modified-id="Producer如何向RabbitMQ发送消息？-2.1.3">Producer如何向RabbitMQ发送消息？</a></span></li><li><span><a href="#Consumer如何接收消息？" data-toc-modified-id="Consumer如何接收消息？-2.1.4">Consumer如何接收消息？</a></span></li><li><span><a href="#RabbitMQ里的queue是什么？" data-toc-modified-id="RabbitMQ里的queue是什么？-2.1.5">RabbitMQ里的queue是什么？</a></span></li><li><span><a href="#Producer发送消息时需要指定的exchange的作用是什么？" data-toc-modified-id="Producer发送消息时需要指定的exchange的作用是什么？-2.1.6">Producer发送消息时需要指定的exchange的作用是什么？</a></span></li><li><span><a href="#RabbitMQ中出现的routing_key的作用是什么？" data-toc-modified-id="RabbitMQ中出现的routing_key的作用是什么？-2.1.7">RabbitMQ中出现的routing_key的作用是什么？</a></span></li><li><span><a href="#binding_key和queue的映射关系是怎么样？" data-toc-modified-id="binding_key和queue的映射关系是怎么样？-2.1.8">binding_key和queue的映射关系是怎么样？</a></span></li><li><span><a href="#exchange如何决定从Producer发送过来的消息放到哪个queue？" data-toc-modified-id="exchange如何决定从Producer发送过来的消息放到哪个queue？-2.1.9">exchange如何决定从Producer发送过来的消息放到哪个queue？</a></span></li><li><span><a href="#RabbitMQ-server重启时怎么处理还在queue里的消息？" data-toc-modified-id="RabbitMQ-server重启时怎么处理还在queue里的消息？-2.1.10">RabbitMQ server重启时怎么处理还在queue里的消息？</a></span></li><li><span><a href="#RabbitMQ怎么确定发送的消息被正确处理了?" data-toc-modified-id="RabbitMQ怎么确定发送的消息被正确处理了?-2.1.11">RabbitMQ怎么确定发送的消息被正确处理了?</a></span></li><li><span><a href="#RabbitMQ如何处理消息的负载均衡的？" data-toc-modified-id="RabbitMQ如何处理消息的负载均衡的？-2.1.12">RabbitMQ如何处理消息的负载均衡的？</a></span></li></ul></li></ul></li><li><span><a href="#参考资料" data-toc-modified-id="参考资料-3">参考资料</a></span></li></ul></div>

# 背景

分布式系统中一个重要组件即消息队列。RabbitMQ是消息队列的一个开源实现。其他的还有RocketMQ、ActiveMQ、Kafka、ZeroMQ、MetaMq。本文记录RabbitMQ的笔记。

# 正文

## 基本概念

### 什么是RabbitMQ？

> RabbitMQ is a message broker: it accepts and forwards messages. 

顾名思义，简单来说，就是一个消息队列，它具备接受消息和转发消息的能力。

最简单的模型如下：

![one-overall](RabbitMQ初探/python-one-overall.png)


其中P代表Producer，产生消息的进程，Producer发送消息到MQ的过程即Producing。C代表Consumer，消费消息的进程，Consumer从MQ接受消息的过程即Consuming。

### 使用RabbitMQ后组件间如何通信？

部署视图中，P，MQ，C没有任何限制，可以同一台host，也可以不同host。简单来讲，就是不同进程。所以组件间的通信本质上是进程间通信，RabbitMQ使用网络进行通信。

Producer发消息前，Consumer获取消息前，都需要先建立到MQ的连接：

```
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
```

一般通过指定主机名或IP地址明确需要连接的MQ server。

### Producer如何向RabbitMQ发送消息？

1. 指定RabbitMQ server的网络地址，打开一条连接

```python
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()
```

2. 声明一个queue

```python
channel.queue_declare(queue='hello')
```

3. 指定exchange, routing_key, 消息体body后发送

```python
channel.basic_publish(exchange='',
                      routing_key='hello',
                      body='Hello World!')
```

### Consumer如何接收消息？

Consumer接收时前面两步类似：
1. 指定RabbitMQ server的网络地址，打开一条连接
2. 声明一个queue（和Producer的声明需要匹配，先声明确保queue存在，否则无法正常接收）
3. 注册接收到消息的回调

```python
def callback(ch, method, properties, body):
    print(" [x] Received %r" % body)

channel.basic_consume(callback,
                      queue='hello',
                      no_ack=True)
```

### RabbitMQ里的queue是什么？

queue简单来说就是一个buffer队列，因为consumer有可能暂时处于busy状态，接到的消息先按照顺序一个一个存在queue里，等consumer可以接收时在从queue里取出来发送给consumer。

### Producer发送消息时需要指定的exchange的作用是什么？

消息不是直接发送到RabbitMQ的queue，而是先发送到RabbitMQ的exchange这个，相当于邮局的概念，起到一个控制器的作用。

![three-overall](RabbitMQ初探/python-three-overall.png)

exchange接收来自Producer的消息，然后把消息放到相应的queue中。

exchange控制消息被放入queue的策略。选择具体策略时需要考虑如下类似问题：

- 放一个queue还是多个queue? 放一个的话具体放到哪个queue? 放多个的话具体放哪几个queue?
- 可不可以直接丢弃?
- 需不要持久化，server重启后没发送的消息还需不需要重新发送?
- 消息发送了怎么确定被正确处理了?
- 如何控制worker queue的负载均衡?

exchange的策略由Producer发送消息时的exchange_type指定。目前可以指定的值包括：

- direct
- fanout
- headers
- topic

（这些值的具体含义后面涉及）

### RabbitMQ中出现的routing_key的作用是什么？


RabbitMQ里多个地方存在routing_key的概念，一个是创建binding时，一个是发送消息是。本质上用来处理消息路由的。

RabbitMQ可以同时创建多个queue：

![three-overall](RabbitMQ初探/python-three-overall.png)

exchange需要控制消息怎么放到queue里，所以Producer可以先在queue和exchange直接创建一个binding。

方法如下：

```python
channel.queue_bind(exchange=exchange_name,
                   queue=queue_name,
                   routing_key='black')
```

创建binding时传递的routing_key一般叫binding key。

简单来说，一个binding就是一个三元组（exchange_name, binding_key, queue_name)。

exchange根据Producer发送的消息里的routing_key来选择queue。具体选择方法由exchange_type决定。

所以，Producer发送消息时需要指定的routing_key的作用相当于消息的地址。

```python
channel.basic_publish(exchange='',
                      routing_key='hello',
                      body='Hello World!')
```

### binding_key和queue的映射关系是怎么样？

简单来说，就是N:N关系。

![N:1](RabbitMQ初探/python-four.png)

值为error的binding_key对多个queue。

info,warn,error多个binding_key对一个queue。

### exchange如何决定从Producer发送过来的消息放到哪个queue？

简单来说，由消息里的exchange_type和routing_key决定。

目前，我们知道，从Producer发送的消息可以看作(exchange_name, exchange_type, routing_key, body)四元组，exchange内部维护(exchange_name, binding_key, queue_name)三元组的路由表。exchange取出消息里的exchange_type和routing_key，对路由表里的binding_key进行匹配。

**direct**

当exchange_type为direct时，exchange使用精确匹配策略，exchange会把消息发给所有binding_key=routing_key的queue。这是exchange的默认行为。

**fanout**

当exchange_type为direct时，exchange把消息转发给所有绑定的queue。

**topic**

当exchange_type为direct时，exchange使用wildcard match在路由表里匹配。`*`代表一个单词，`#`代表零个或多个单词。

**headers**

当exchange_type为direct时, exchange使用消息的header进行匹配。

### RabbitMQ server重启时怎么处理还在queue里的消息？

默认情况下，RabbitMQ server重启会丢失之前queue里没处理完的消息。可以在声明queue时使用durable queue：

```python
channel.queue_declare(queue='hello', durable=True)
```

然后Producer发送消息时使用persistent mode：

```python
channel.basic_publish(exchange='',
                      routing_key="task_queue",
                      body=message,
                      properties=pika.BasicProperties(
                         delivery_mode = 2, # make message persistent
                      ))
```

### RabbitMQ怎么确定发送的消息被正确处理了?

Consumer处理消息的过程可能会很长，可能处理过程Consumer crash了。所以RabbitMQ需要一种确保消息被Consumer正确处理的反馈机制。RabbitMQ server把消息转发到给Consumer后的queue后，会等到Consumer的ack信号，如果没有等到但是channel关闭或连接关闭或TCP连接断开，该消息会被重新排队分派给处于活动状态的worker。

### RabbitMQ如何处理消息的负载均衡的？

负载均衡由queue处理，一个queue可以挂多个consumer，但一个consumer只能监听一个queue。默认情况下，RabbitMQ会对挂在同一个queue下的worker应用程序使用round-robbin（轮训）进行消息分发。Consumer可以在自己的channel上设置一次处理一个消息，避免过载太大。

# 参考资料

- [RabbitMQ Get Started](https://www.rabbitmq.com/getstarted.html)